# Concurrency models: Actor, STM and CSP

---

## Financial Team TWIL Shared

> Shared by [laserx](https://github.com/laserx)

## References

* [Clojure offical guide: Refs and Transactions](https://clojure.org/reference/refs)
* [The actor model in 10 minutes](https://www.brianstorti.com/the-actor-model/)
* [Elixir offical guide: Processes](https://elixir-lang.org/getting-started/processes.html#spawn)
* [Overview of Modern Concurrency and Parallelism Concepts
](https://nikolaygrozev.wordpress.com/2015/07/14/overview-of-modern-concurrency-and-parallelism-concepts/)
* [Visualizing Concurrency in Go](https://divan.github.io/posts/go_concurrency_visualize/)
* [Locks, Actors, And Stm In Pictures](http://adit.io/posts/2013-05-15-Locks,-Actors,-And-STM-In-Pictures.html)
* [Dining philosophers problem](https://en.wikipedia.org/wiki/Dining_philosophers_problem)
* [Dining philosophers](http://rosettacode.org/wiki/Dining_philosophers)
---

## Actor model

The actor model adopts the philosophy that everything is an actor. This is similar to the everything is an object philosophy used by some object-oriented programming languages.

An actor is a computational entity that, in response to a message it receives, can concurrently:

send a finite number of messages to other actors;
create a finite number of new actors;
designate the behavior to be used for the next message it receives.
There is no assumed sequence to the above actions and they could be carried out in parallel.

Decoupling the sender from communications sent was a fundamental advance of the Actor model enabling asynchronous communication and control structures as patterns of passing messages.

Recipients of messages are identified by address, sometimes called "mailing address". Thus an actor can only communicate with actors whose addresses it has. It can obtain those from a message it receives, or if the address is for an actor it has itself created.

The actor model is characterized by inherent concurrency of computation within and among actors, dynamic creation of actors, inclusion of actor addresses in messages, and interaction only through direct asynchronous message passing with no restriction on message arrival order.

```elixir
defmodule Calc do
  def gen(times), do: for(i <- 1..times, do: i)

  def split(enumerable) do
    enumerable
    |> Enum.split(enumerable |> length() |> div(2))
  end

  def sum(enumerable, pid) do
    spawn(fn -> send(pid, {:value, Enum.sum(enumerable)}) end)
  end

  def receiver() do
    receive do
      {:value, value} ->
        IO.inspect(value)
    after
      1_000 ->
        IO.puts("nothing after 1s")
        Process.exit(self(), :normal)
    end

    receiver()
  end
end

{x, y} = Calc.gen(10000) |> Calc.split()

Calc.sum(x, self())
Calc.sum(y, self())

Calc.receiver()
```

---

## STM (Software Transactional Memory)

In computer science, software transactional memory (STM) is a concurrency control mechanism analogous to database transactions for controlling access to shared memory in concurrent computing. It is an alternative to lock-based synchronization. STM is a strategy implemented in software, rather than as a hardware component. A transaction in this context occurs when a piece of code executes a series of reads and writes to shared memory. These reads and writes logically occur at a single instant in time; intermediate states are not visible to other (successful) transactions.

Clojure transactions should be easy to understand if you’ve ever used database transactions - they ensure that all actions on Refs are atomic, consistent, and isolated. Atomic means that every change to Refs made within a transaction occurs or none do. Consistent means that each new value can be checked with a validator function before allowing the transaction to commit. Isolated means that no transaction sees the effects of any other transaction while it is running. Another feature common to STMs is that, should a transaction have a conflict while running, it is automatically retried.

---

## CSP (Communicating Sequential Processes)

Communicating Sequential Processes (CSP) is a paradigm which is very similar to the Actor model. It’s also based on message-passing without sharing memory. However, CSP and Actors have these 2 key differences:

* Processes in CSP are anonymous, while actors have identities. So, CSP uses explicit channels for message passing, whereas with Actors you send messages directly.
* With CSP the sender cannot transmit a message until the receiver is ready to accept it. Actors can send messages asynchronously (e.g. with async calls in Celluloid).

CSP is implemented in such programming languages as Go with goroutines and channels, Clojure with the core.async library and Crystal with fibers and channels.


code example:

```go
package main

import "fmt"

func sum(s []int, c chan int) {
	sum := 0
	for _, v := range s {
		sum += v
	}
	c <- sum // send sum to c
}

func gen(t int) []int {
	var r []int
	for i := 1; i <= t; i++ {
		r = append(r, i)
	}

	return r
}

func main() {
	s := gen(10000)

	fmt.Println(s)

	c := make(chan int)
	go sum(s[:len(s)/2], c)
	go sum(s[len(s)/2:], c)
	x, y := <-c, <-c // receive from c

	fmt.Println(x, y, x+y)
}

```

---

## Question

Dining philosophers problem

Five philosophers, Aristotle, Kant, Spinoza, Marx, and Russell (the tasks) spend their time thinking and eating spaghetti. They eat at a round table with five individual seats. For eating each philosopher needs two forks (the resources). There are five forks on the table, one left and one right of each seat. When a philosopher cannot grab both forks it sits and waits. Eating takes random time, then the philosopher puts the forks down and leaves the dining room. After spending some random time thinking about the nature of the universe, he again becomes hungry, and the circle repeats itself.

It can be observed that a straightforward solution, when forks are implemented by semaphores, is exposed to deadlock. There exist two deadlock states when all five philosophers are sitting at the table holding one fork each. One deadlock state is when each philosopher has grabbed the fork left of him, and another is when each has the fork on his right.